In [1]:
# Clonning yolov5 repo to our google colab workspace
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 15921, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 15921 (delta 16), reused 24 (delta 9), pack-reused 15880
Receiving objects: 100% (15921/15921), 14.60 MiB | 22.34 MiB/s, done.
Resolving deltas: 100% (10915/10915), done.


In [ ]:
# install dependencies as necessary
%cd yolov5
!pip install -qr requirements.txt
import torch
from IPython.display import Image, clear_output  # to display images
# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
#Here I used furniture-ngpea dataset which consist of 161 images of chairs, tables and sofas combines
# Download Dataset in yolov5 directory using roboflow package
%cd /content/yolov5
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="bys15vogVKtmTbB1lhxM")
project = rf.workspace("roboflow-100").project("furniture-ngpea")
dataset = project.version(2).download("yolov5")

In [4]:
# this is the YAML file with our data
%cat {dataset.location}/data.yaml

names:
- Chair
- Sofa
- Table
nc: 3
train: furniture-2/train/images
val: furniture-2/valid/images


In [5]:
# Loading the model we trained on furnitures dataset using yolov5 earliar
model = torch.hub.load('/content/yolov5/', 'custom', path='/content/custom_weight_yolov5/best.pt', source='local', force_reload = True)
model.conf = 0.4

YOLOv5 🚀 v7.0-210-gdd10481 Python-3.10.12 torch-2.0.1+cu118 CPU



[Errno 2] No such file or directory: '/usr/local/lib/python3.10/dist-packages/pyparsing-3.1.1.dist-info/METADATA'


Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7251912 parameters, 0 gradients
Adding AutoShape... 


In [6]:
import os
import cv2
from google.colab.patches import cv2_imshow
image_directory = '/content/yolov5/furniture-2/valid/images'
labels_directory = '/content/yolov5/furniture-2/valid/labels'
labels = {0: "chair", 1: "sofa", 2: "table"} # Labels dictionary
global FALSE_PREDICTION
FALSE_PREDICTION = 0 # Number of wrong predictions

In [7]:
# creating a dir to store fp and fn of all classes respectively
import os
for i in labels:
  path1 = f"/content/diff_fp_fn/{labels[i]}/fp"
  path2 = f"/content/diff_fp_fn/{labels[i]}/fn"
  if not os.path.exists(path1):
    os.makedirs(path1)
  if not os.path.exists(path2):
    os.makedirs(path2)


In [8]:
def save_img(category_path, img1, img2):
  # Combine img1 and img2 and save them in respective directory
  new_img = cv2.hconcat([img1, img2])
  cv2_imshow(new_img)
  cv2.imwrite("/content/diff_fp_fn/"+category_path+".jpg", new_img)

def calculate_iou(box1, box2):
    # Returns Intersection over Union of two boxes
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    intersection_x1 = max(x1_min, x2_min)
    intersection_y1 = max(y1_min, y2_min)
    intersection_x2 = min(x1_max, x2_max)
    intersection_y2 = min(y1_max, y2_max)

    intersection_area = max(0, intersection_x2 - intersection_x1 + 1) * max(0, intersection_y2 - intersection_y1 + 1)

    box1_area = (x1_max - x1_min + 1) * (y1_max - y1_min + 1)
    box2_area = (x2_max - x2_min + 1) * (y2_max - y2_min + 1)

    iou = intersection_area / float(box1_area + box2_area - intersection_area)

    return iou

def evaluate_image(img_name):
  # Partition of FP and FN images by classes

  global FALSE_PREDICTION
  image_path = os.path.join(image_directory, img_name+".jpg")
  labels_path = os.path.join(labels_directory, img_name+".txt")
  with open(labels_path) as f:
    lines = f.readlines()
    gt = [float(x) for x in lines[0].split()]

  img_gt = cv2.imread(image_path)
  img_pred = cv2.imread(image_path)

  pred = model(image_path)
  plabel, pc, pconf = pred.xyxy[0].cpu().numpy()[:, -1], pred.xyxy[0].cpu().numpy()[:, :-2], pred.xyxy[0].cpu().numpy()[:, -2]
  gt_label, gt_c = gt[0], gt[1:]
  gtx1 = int(gt_c[0]*1200) - int(gt_c[2]*1200/2)
  gty1 = int(gt_c[1]*1200) - int(gt_c[3]*1200/2)
  gtx2 = int(gt_c[0]*1200) + int(gt_c[2]*1200/2)
  gty2 = int(gt_c[1]*1200) + int(gt_c[3]*1200/2)

  if len(plabel) > 1:
    # if more than one predicted class
    FALSE_PREDICTION+=1
    for i in range(len(plabel)):
      px1 = int(pc[i][0])
      py1 = int(pc[i][1])
      px2 = int(pc[i][2])
      py2 = int(pc[i][3])
      img_pred = cv2.rectangle(img_pred, (px1, py1), (px2, py2), (255, 0, 0), 5)
      img_pred = cv2.putText(img_pred, f"Prediction: {labels[plabel[i]].upper()}({'{:.3f}'.format(pconf[i])})", (px1, py1+50), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 225, 0), 2)

    img_gt = cv2.rectangle(img_gt, (gtx1, gty1), (gtx2, gty2), (255, 0, 0), 5)
    img_gt = cv2.putText(img_gt, f"Ground Truth: {labels[gt_label].upper()}", (gtx1, gty1 + 50), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 225, 0), 2)

    for i in range(len(plabel)):
      px1 = int(pc[i][0])
      py1 = int(pc[i][1])
      px2 = int(pc[i][2])
      py2 = int(pc[i][3])

      if plabel[i]!=gt_label:
        path = f"{labels[plabel[i]]}/fp/{img_name}"
        save_img(path, img_gt, img_pred)

      else:
        if calculate_iou((px1, py1, px2, py2), (gtx1, gty1, gtx2, gty2)) < 0.5:
          img_p = cv2.imread(image_path)
          for i in range(len(plabel)):
            if plabel[i]!=gt_label:
              px1 = int(pc[i][0])
              py1 = int(pc[i][1])
              px2 = int(pc[i][2])
              py2 = int(pc[i][3])
              img_p = cv2.rectangle(img_p, (px1, py1), (px2, py2), (255, 0, 0), 5)
              img_p = cv2.putText(img_p, f"Prediction: {labels[plabel[i]].upper()}({'{:.3f}'.format(pconf[i])})", (px1, py1+50), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 225, 0), 2)

          path = f"{labels[gt_label]}/fn/{img_name}"
          save_img(path, img_gt, img_p)

    if gt_label not in plabel:
      path = f"{labels[gt_label]}/fn/{img_name}"
      save_img(path, img_gt, img_pred)

  elif len(plabel) == 0:
    # if no prediction by the model
    FALSE_PREDICTION+=1
    path = f"{labels[gt_label]}/fn/{img_name}"
    img_gt = cv2.rectangle(img_gt, (gtx1, gty1), (gtx2, gty2), (255, 0, 0), 5)
    img_gt = cv2.putText(img_gt, f"Ground Truth: {labels[gt_label].upper()}", (gtx1, gty1 + 50), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 225, 0), 2)
    save_img(path, img_gt, img_pred)

  elif gt_label!=plabel[0]:
    # if image is missclassified by image
    FALSE_PREDICTION+=1
    px1 = int(pc[0][0])
    py1 = int(pc[0][1])
    px2 = int(pc[0][2])
    py2 = int(pc[0][3])
    img_pred = cv2.rectangle(img_pred, (px1, py1), (px2, py2), (255, 0, 0), 5)
    img_pred = cv2.putText(img_pred, f"Prediction: {labels[plabel[0]].upper()}({'{:.3f}'.format(pconf[0])})", (px1, py1+50), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 225, 0), 2)
    img_gt = cv2.rectangle(img_gt, (gtx1, gty1), (gtx2, gty2), (255, 0, 0), 2)
    img_gt = cv2.putText(img_gt, f"Ground Truth: {labels[gt_label].upper()}", (gtx1, gty1 + 50), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 225, 0), 2)
    path1 = f"{labels[plabel[0]]}/fp/{img_name}"
    path2 = f"{labels[gt_label]}/fn/{img_name}"
    save_img(path1, img_gt, img_pred)
    save_img(path2, img_gt, img_pred)

  else:
    px1 = int(pc[0][0])
    py1 = int(pc[0][1])
    px2 = int(pc[0][2])
    py2 = int(pc[0][3])
    img_gt = cv2.rectangle(img_gt, (gtx1, gty1), (gtx2, gty2), (255, 0, 0), 5)
    img_gt = cv2.putText(img_gt, f"Ground Truth: {labels[gt_label].upper()}", (gtx1, gty1 + 50), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 225, 0), 2)
    if calculate_iou((px1, py1, px2, py2), (gtx1, gty1, gtx2, gty2)) < 0.5:
      # if iou value is < 0.5 of ground truh and predicted image
      FALSE_PREDICTION+=1
      path = f"{labels[gt_label]}/fn/{img_name}"
      save_img(path, img_gt, cv2.imread(image_path))

In [ ]:
# Loop over validation dir and evaluate each image individually
for filename in os.listdir(image_directory):
    evaluate_image(filename.split(".jpg")[0])

In [10]:
print(f"NO of Images in the Validation SET: 161")
print(f"No of false predictions: {FALSE_PREDICTION}")

NO of Images in the Validation SET: 161
No of false predictions: 27
